In [1]:
import pickle
import numpy as np
import yaml

In [2]:
# Import specified definitions only from given notebook
import ipynb.fs
from .defs.shared_constants import MISSING_INT, MISSING_FLOAT

from .defs.shared_auxiliary import draw_disturbances
from .defs.read import read_init_file
from .defs.read import init_dict_to_attr_dict

from .defs.shared_auxiliary import calculate_wage_systematic
from .defs.shared_auxiliary import calculate_period_wages
from .defs.shared_auxiliary import calculate_consumption_utilities
from .defs.shared_auxiliary import calculate_total_utilities
from .defs.shared_auxiliary import calculate_utilities
from .defs.shared_auxiliary import calculate_continuation_values

In [3]:
# Read in initialization file as attr_dict
attr_dict = read_init_file('toy_model_init_file.yml')

In [4]:
# Import the final output of pyth_create_state_space, args
file_name = "args_file.pkl"
# Open the file for reading
file_object = open(file_name,'rb')  
# load the object from the file into var args
state_space_args = pickle.load(file_object)

In [5]:
def construct_covariates(attr_dict, states_all, period, k):
    """Constructs additional covariates given state space components."""
    
    # Determine education level given number of years of education
    # Would it be more efficient to do this somewhere else?
    
    # Unpack attributes from the model specification:
    educ_min = attr_dict['INITIAL_CONDITIONS']['educ_min']
    
    # Unpack state space components
    educ_years = states_all[period, k, 0]

    # Extract education information
    if (educ_years <= 10):
        educ_level = [1,0,0]

    elif (educ_years > 10) and (educ_years <= 12):
        educ_level = [0,1,0]

    else:
        educ_level = [0,0,1]

    educ_years_idx = educ_years - educ_min
    
    # Return function output
    return educ_level, educ_years_idx

def construct_emax (attr_dict,
                    period,
                    k,
                    educ_level,
                    educ_years_idx,
                    num_draws_emax,
                    draws_emax_period,
                    states_all,
                    mapping_states_index,
                    optim_paras,
                    periods_emax,
):
    """Obtain the maximum of the value fucntion over the available choices 
    via a Monte Carlo simulation integration procedure.
    """
    
    # Unpack attributes from the model specification:
    delta = attr_dict['CONSTANTS']['delta']
    
    # Initialize container for sum of maximum value functions
    # over all error term draws for the period and state
    emax = 0.0
    
    # Loop over all error term draws
    # for the period and state currently rached by the parent loop
    for i in range(num_draws_emax):
        
        # Extract the error term draws corresponding to
        # period number, state, and loop iteration number, i
        corresponding_draws = draws_emax_period[i, :]
        
        # Extract relevant state space components 
        educ_years, _, exp_p, exp_f = states_all[period, k, :]
        
        # Calculate flow utility at current period, state, and draw
        flow_utilities = calculate_utilities(attr_dict, educ_level, exp_p, exp_f, optim_paras, corresponding_draws)[0]
        
        # Obtain continuation values for all choices
        continuation_values = calculate_continuation_values(attr_dict, mapping_states_index, periods_emax, period, educ_years_idx, exp_p, exp_f)
        
        # Calculate choice specific value functions
        value_functions = flow_utilities + delta*continuation_values
        
        # Obtain highest value function
        maximum = max(value_functions)
        
        # Add to sum over all draws
        emax += maximum
        
        # End loop
    
    # Average over the number of draws
    emax = emax / num_draws_emax
    
    # Thus, we have integrated out the error term
    
    # Output
    return emax

In [6]:
def pyth_backward_induction(attr_dict, state_space_args):
    """Solves the model in a backward induction procedure."""
    
    # Unpack objects from agrs
    states_all, states_number_period, mapping_states_index, max_states_period = state_space_args[0], state_space_args[1], state_space_args[2], state_space_args[3]
    
    # Unpack parameter from the model specification
    num_periods = attr_dict['GENERAL']['num_periods']
    num_draws_emax = attr_dict['SOLUTION']['num_draws_emax']
    seed_emax = attr_dict['SOLUTION']['seed_emax']
    shocks_cov = attr_dict['DERIVED_ATTR']['shocks_cov']
    optim_paras = attr_dict['PARAMETERS']['optim_paras']

    # Initialize container for the final result,
    # maximal value function per perdiod and state:
    periods_emax = np.tile(MISSING_FLOAT, (num_periods, max_states_period))

    draws_emax = draw_disturbances((num_periods, num_draws_emax), shocks_cov, seed_emax)

    # Loop over all periods
    for period in range(num_periods - 1, -1, -1):

        # Select the random draws for Monte Carlo integration relevant for the period
        draws_emax_period = draws_emax[period, :, :]

        # Loop over all admissible state space points
        # for the period currently reached by the parent loop
        for k in range(states_number_period[period]):

            # Construct additional education information
            educ_level, educ_years_idx = construct_covariates(attr_dict, states_all, period, k)  

            # Integrate out the error term
            emax = construct_emax (
                attr_dict,
                period,
                k,
                educ_level,
                educ_years_idx,
                num_draws_emax,
                draws_emax_period,
                states_all,
                mapping_states_index,
                optim_paras,
                periods_emax,
            )

            # Record function output
            periods_emax[period, k] = emax
        
    # Return output
    return periods_emax

In [7]:
periods_emax = pyth_backward_induction(attr_dict, state_space_args)

Export final output:

In [8]:
# Choose a file name
file_name = "periods_emax_file.pkl"

# Open the file for writing
with open(file_name,'wb') as my_file_obj:
    pickle.dump(periods_emax, my_file_obj)  